In [1]:
import sys
import os
sys.path.append(os.path.abspath("../src"))

In [3]:
from cot_reports import cot_all_reports
cot_data_tuple = cot_all_reports(store_txt=True, verbose=True)
legacy_fut = cot_data_tuple[0]

legacy_fut
Selected: COT Legacy report. Futures only.
Stored the extracted file FUT86_16.txt in the working directory.
Selected: legacy_fut
Downloaded single year data from: 2017
Stored the file annual.txt in the working directory.
Selected: legacy_fut
Downloaded single year data from: 2018
Stored the file annual.txt in the working directory.
Selected: legacy_fut
Downloaded single year data from: 2019
Stored the file annual.txt in the working directory.
Selected: legacy_fut
Downloaded single year data from: 2020
Stored the file annual.txt in the working directory.
Selected: legacy_fut
Downloaded single year data from: 2021
Stored the file annual.txt in the working directory.
Selected: legacy_fut
Downloaded single year data from: 2022
Stored the file annual.txt in the working directory.
Selected: legacy_fut
Downloaded single year data from: 2023
Stored the file annual.txt in the working directory.
Selected: legacy_fut
Downloaded single year data from: 2024
Stored the file annual.txt in 

In [2]:
from utils.data_loader import load_cot_data

macro = load_cot_data(symbol="DX1!",years= 2, legacy_fut=legacy_fut)
print(macro)

NameError: name 'legacy_fut' is not defined

In [3]:
from utils.data_loader import load_asset_data
eurfuture = load_asset_data(symbol="DX1!", years= 2)

KeyboardInterrupt: 

In [9]:
eurfuture

{'1D_data':                          time     open     high      low    close   volume
 0   2023-08-09 07:00:00+07:00  102.350  102.405  102.120  102.312   8116.0
 1   2023-08-10 07:00:00+07:00  102.330  102.500  101.600  102.367  17705.0
 2   2023-08-11 07:00:00+07:00  102.490  102.765  102.265  102.689  17970.0
 3   2023-08-14 05:00:00+07:00  102.715  103.325  102.630  103.058  16466.0
 4   2023-08-15 07:00:00+07:00  103.005  103.160  102.680  103.095  18642.0
 ..                        ...      ...      ...      ...      ...      ...
 499 2025-07-16 07:00:00+07:00   98.290   98.595   97.395   98.075  41651.0
 500 2025-07-17 07:00:00+07:00   98.055   98.665   98.040   98.446  15627.0
 501 2025-07-18 07:00:00+07:00   98.215   98.295   97.815   98.198  11997.0
 502 2025-07-21 05:00:00+07:00   98.120   98.235   97.425   97.575  20031.0
 503 2025-07-22 07:00:00+07:00   97.570   97.725   97.495   97.555   4131.0
 
 [504 rows x 6 columns],
 '1W_data':                          time     open

In [4]:
from utils.data_loader import load_economic_data

In [5]:
result = load_economic_data("EURUSD", 1)
print(result)
print(type(result))

Loading GDP_Growth data with symbol: ECONOMICS:EUGDPYY, TF: 3M
Loading Interest_Rate data with symbol: ECONOMICS:EUINTR, TF: 1M
Loading Inflation_Rate data with symbol: ECONOMICS:EUIRYY, TF: 1M
Loading CPI data with symbol: ECONOMICS:EUCPI, TF: 1M
Loading PPI data with symbol: ECONOMICS:EUPPI, TF: 1M
Loading Unemployment data with symbol: ECONOMICS:EUUR, TF: 1M
Loading Trade_Balance data with symbol: ECONOMICS:EUBOT, TF: 1M
Loading Gov_Debt data with symbol: ECONOMICS:EUGDG, TF: 12M
Loading Consumer_Confidence data with symbol: ECONOMICS:EUCCI, TF: 1M
Loading Retail_Sales data with symbol: ECONOMICS:EURSMM, TF: 1M
Loading Money_Supply data with symbol: ECONOMICS:EUM2, TF: 1M
{'GDP_Growth':                         time  open  high  low  close
0  2022-04-01 07:00:00+07:00   4.1   4.1  4.1    4.1
1  2022-07-01 07:00:00+07:00   2.5   2.5  2.5    2.5
2  2022-10-01 07:00:00+07:00   1.9   1.9  1.9    1.9
3  2023-01-01 07:00:00+07:00   1.3   1.3  1.3    1.3
4  2023-04-01 07:00:00+07:00   0.6  

In [7]:
result['GDP_Growth']

,time,open,high,low,close
0,2022-04-01 07:00:00+07:00,4.1,4.1,4.1,4.1
1,2022-07-01 07:00:00+07:00,2.5,2.5,2.5,2.5
2,2022-10-01 07:00:00+07:00,1.9,1.9,1.9,1.9
3,2023-01-01 07:00:00+07:00,1.3,1.3,1.3,1.3
4,2023-04-01 07:00:00+07:00,0.6,0.6,0.6,0.6
5,2023-07-01 07:00:00+07:00,0.1,0.1,0.1,0.1
6,2023-10-01 07:00:00+07:00,0.2,0.2,0.2,0.2
7,2024-01-01 07:00:00+07:00,0.5,0.5,0.5,0.5
8,2024-04-01 07:00:00+07:00,0.5,0.5,0.5,0.5
9,2024-07-01 07:00:00+07:00,1.0,1.0,1.0,1.0


In [4]:
from utils.data_loader import load_asset_data

btc = load_asset_data(symbol="BTCUSD", years= 1)

In [9]:
from utils.data_loader import load_portfolio_data

btc = load_portfolio_data(category="crypto", years= 1, legacy_fut=legacy_fut)

⏳ Loading data for BTCUSD...
❌ Failed to load economic data for BTCUSD: Symbol 'BTCUSD' is not supported in country_map.
⏳ Loading data for ETHUSD...
❌ Failed to load economic data for ETHUSD: Symbol 'ETHUSD' is not supported in country_map.
⏳ Loading data for LTCUSD...
❌ Failed to load economic data for LTCUSD: Symbol 'LTCUSD' is not supported in country_map.
⚠️ Symbol LTCUSD is not available in the COT mapping list.
⏳ Loading data for SOLUSD...
❌ Failed to load economic data for SOLUSD: Symbol 'SOLUSD' is not supported in country_map.
⚠️ Symbol SOLUSD is not available in the COT mapping list.
⏳ Loading data for XRPUSD...
❌ Failed to load economic data for XRPUSD: Symbol 'XRPUSD' is not supported in country_map.
⚠️ Symbol XRPUSD is not available in the COT mapping list.


In [10]:
btc

{'BTCUSD': {'asset_data': {'1D_data':                          time       open       high        low      close
   0   2024-11-14 07:00:00+07:00   90459.32   91859.45   86786.94   87424.56
   1   2024-11-15 07:00:00+07:00   87425.27   91832.07   87087.19   91034.61
   2   2024-11-16 07:00:00+07:00   91034.58   91776.58   90097.03   90581.65
   3   2024-11-17 07:00:00+07:00   90583.12   91473.85   88753.71   89876.59
   4   2024-11-18 07:00:00+07:00   89878.14   92586.29   89395.29   90473.71
   ..                        ...        ...        ...        ...        ...
   247 2025-07-19 07:00:00+07:00  118006.64  118570.41  117367.23  117920.27
   248 2025-07-20 07:00:00+07:00  117920.27  118909.59  116553.53  117328.31
   249 2025-07-21 07:00:00+07:00  117329.48  119675.96  116573.34  117430.98
   250 2025-07-22 07:00:00+07:00  117430.99  120296.21  116193.29  120008.72
   251 2025-07-23 07:00:00+07:00  120008.86  120138.50  118580.48  119016.34
   
   [252 rows x 5 columns],
   '1W_dat